In [ ]:
__author__ = 'Vinicius Placco <vinicius.placco@noirlab.edu>'
__version__ = '20240125' # yyyymmdd; version datestamp of this notebook
#__datasets__ = ['allwise','smash']
__keywords__ = ['gnirs','gemini','GRB','dragons']

# Gemini GNIRS reduction using DRAGONS Python API
***
## Public archival data from gnirsimg_tutorial - GN-2011B-Q-34 (GRB120116A)
#### adapted from https://dragons.readthedocs.io/projects/gnirsimg-drtutorial/en/v3.1.0/01_overview.html
#### don't forget to `conda install -n dragons nb_conda_kernels ipykernel` to run this notebook on the DRAGONS env 
***

## Table of contents
* [Goals](#goals)
* [Summary](#summary)
* [Disclaimers and attribution](#disclaimer)
* [Imports and setup](#imports)
* [About the dataset](#About)
* [Downloading data for reduction](#Downloading_Data)
* [Set up the DRAGONS logger](#DRAGONS_logger)
* [Create File Lists](#File_Lists)
* [Create Master Dark](#Master_dark)
* [Create Master Flat Field](#Master_Flat)
* [Reduce Science Images](#Reduce_Science)
* [Display stacked final image](#Display_Image)
* [Clean-up (optional)](#Clean-up)

<a class="anchor" id="goals"></a>
# Goals
Showcase how to perform GNIRS keyhole imaging data reduction using the Gemini DRAGONS package on the Data Lab science platform. Uses a custom DRAGONS kernel `"DRAGONS (Py3.7)"`. The steps include downloading data from the Gemini archive, setting up a DRAGONS calibration service, processing of flats, darks, and science frames, and finally the creation of a single combined stacked image.

<a class="anchor" id="summary"></a>
# Summary
DRAGONS is a Python-based astronomical data reduction platform written by the Gemini Science User Support Department. It currently can be used to reduce imaging data from Gemini instruments GMOS, NIRI, Flamingos 2, GSAOI, and GNIRS, and spectroscopic data in GMOS longslit mode. Linked here is a general list of guides, manuals, and tutorials about the use of DRAGONS:
https://dragons.readthedocs.io/en/v3.1.0/

The DRAGONS kernel has been made available in the Data Lab environment, which should allow users to access the routines without being dependent on installing the software in their local machines. 

In this notebook, we present an example of a DRAGONS Jupyter notebook that works in the Data Lab environment to fully reduce example Gemini North GNIRS J-band imaging data.
This is a version of the DRAGONS Jupyter notebook tutorial presented here: 
https://dragons.readthedocs.io/projects/gnirsimg-drtutorial/en/v3.1.0/ex1_gnirsim_twostars_api.html

This notebook will not present all of the details of the many options available to adjust or optimize the DRAGONS GNIRS data reduction process, rather will just show one example of a standard reduction of a GNIRS imaging dataset. 
More extensive explanations can be found in the general DRAGONS GNIRS data reduction tutorial from Gemini linked here:
https://dragons.readthedocs.io/projects/gnirsimg-drtutorial/en/v3.1.0/index.html

The data used in this notebook example is GNIRS J-band keyhole imaging from the Gemini archive of the gamma-ray burst GRB120116A from the Gemini North program "Investigating gamma-ray bursts and their use as cosmological probes", PI: Nial Tanvir, program ID GN-2011B-Q-34. More program information is given here: https://archive.gemini.edu/programinfo/GN-2011B-Q-34.

<a class="anchor" id="disclaimer"></a>
# Disclaimer & attribution
If you use this notebook for your published science, please acknowledge the following:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, http://dx.doi.org/10.1117/12.2057445

* Data Lab disclaimer: https://datalab.noirlab.edu/disclaimers.php

* DRAGONS publication: Labrie et al., "DRAGONS - Data Reduction for Astronomy from Gemini Observatory North and South", ASPC, 523, 321L, https://ui.adsabs.harvard.edu/abs/2019ASPC..523..321L/abstract

* DRAGONS open source software publication: https://zenodo.org/record/7776065#.ZDg5qOzMLUI


<a class="anchor" id="imports"></a>
# Importing Python Libraries** (you'll probably have to install the `wget` and `ipympl` libraries)

In [ ]:
from __future__ import print_function

import glob
import wget

#DRAGONS
from recipe_system import cal_service
from recipe_system.reduction.coreReduce import Reduce
from gempy.adlibrary import dataselect
from gempy.utils import logutils
from recipe_system.utils.reduce_utils import normalize_ucals

from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cbook as cbook
from matplotlib.colors import LogNorm
from matplotlib.colors import Normalize
from matplotlib.colors import PowerNorm

from collections import namedtuple

#%matplotlib widget
%matplotlib inline

<a class="anchor" id="About"></a>
# About the dataset

This is a GNIRS acquisition keyhole imaging observation of a point source. The observation sequence uses dither-on-target. Dithered observations nearby in time will be used for sky subtraction of each frame.

The calibrations we use for this example include:

- Darks for the science frames
- Flats, as a sequence of lamps-on and lamps-off exposures

| Observation Type | File name(s) | Purpose and Exposure (seconds) |
| :--- | :--- | :---: |
| Science | N20120117S0014-33 | J-band, on-target |
| Science darks | N20120102S0538-547 | 60 s, like Science |
| Flats | N20120117S0034-41 | lamps-on |
| Flats | N20120117S0042-49 | lamps-off |

A note about finding the darks in the GOA. Since GNIRS is not an imager and imaging through the keyhole is done only in extreme circumstances, the archive does not have calibration association rules for the darks in this case. One needs to manually search for the darks. Here is the search that was done to find the darks for this observation sequence:

- Set a date range around the dates of the science observations. In this case we used “20120101-20120131”.
- Set Instrument to GNIRS.
- Set Obs.Type to DARK.
- Set the exposure time to 60 seconds.
- On the result table, select the darks with a “Pass” setting in the “QA” column.

<a class="anchor" id="Downloading_Data"></a>
# Downloading the data (direct link to .tar file hosted at Gemini - 35 Megabytes)

In [ ]:
# uncomment the line and run the cell
#
#wget.download("http://www.gemini.edu/sciops/data/software/datapkgs/gnirsimg_tutorial_datapkg-v1.tar")

**Downloading the data** (individual files - direct link to the Gemini Archive - 184 Megabytes)

In [ ]:
# # uncomment the lines and run the cell

# # Science 
#wget.download("http://archive.gemini.edu/file/N20120117S0014.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0015.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0016.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0017.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0018.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0019.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0020.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0021.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0022.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0023.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0024.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0025.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0026.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0027.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0028.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0029.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0030.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0031.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0032.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0033.fits")

# # Science darks
#wget.download("http://archive.gemini.edu/file/N20120102S0538.fits")
#wget.download("http://archive.gemini.edu/file/N20120102S0539.fits")
#wget.download("http://archive.gemini.edu/file/N20120102S0540.fits")
#wget.download("http://archive.gemini.edu/file/N20120102S0541.fits")
#wget.download("http://archive.gemini.edu/file/N20120102S0542.fits")
#wget.download("http://archive.gemini.edu/file/N20120102S0543.fits")
#wget.download("http://archive.gemini.edu/file/N20120102S0544.fits")
#wget.download("http://archive.gemini.edu/file/N20120102S0545.fits")
#wget.download("http://archive.gemini.edu/file/N20120102S0546.fits")
#wget.download("http://archive.gemini.edu/file/N20120102S0547.fits")

# # Flats (lamp-on)
#wget.download("http://archive.gemini.edu/file/N20120117S0034.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0035.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0036.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0037.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0038.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0039.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0040.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0041.fits")

# # Flats (lamp-off)
#wget.download("http://archive.gemini.edu/file/N20120117S0042.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0043.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0044.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0045.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0046.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0047.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0048.fits")
#wget.download("http://archive.gemini.edu/file/N20120117S0049.fits")

# Create and move data to raw/ directory (uncomment first)

In [ ]:
#!mkdir raw/
#!mv N2012*.fits raw/

In [ ]:
# Check header of one raw science image

tmp = fits.open("raw/N20120117S0014.fits")
tmp[0].header

<a class="anchor" id="DRAGONS_logger"></a>
# Setting up the DRAGONS logger

DRAGONS comes with a local calibration manager that uses the same calibration association rules as the Gemini Observatory Archive. This allows reduce to make requests to a local light-weight database for matching processed calibrations when needed to reduce a dataset.

This simply tells the system where to put the calibration database, the database that will keep track of the processed calibrations we are going to send to it.

In [ ]:
logutils.config(file_name='gnirs_data_reduction.log')

**Create a list of all the FITS files in the directory**

In [ ]:
all_files = glob.glob('raw/N2012*[0-9].fits')
all_files.sort()
#all_files

<a class="anchor" id="File_Lists"></a>
# Create file lists

This data set contains science and calibration frames. For some programs, it could have different observed targets and different exposure times depending on how you like to organize your raw data.

The DRAGONS data reduction pipeline does not organize the data for you. You have to do it. DRAGONS provides tools to help you with that.

The first step is to create lists that will be used in the data reduction process. For that, we use dataselect. Please, refer to the dataselect documentation for details regarding its usage.

In [ ]:
darks60 = dataselect.select_data(
    all_files,
    ['DARK'],
    [],
    dataselect.expr_parser('exposure_time==60')
)

flats = dataselect.select_data(all_files, ['FLAT'])

target = dataselect.select_data(
    all_files,
    [],
    [],
    dataselect.expr_parser('object=="GRB120116A"')
)

<a class="anchor" id="Master_dark"></a>
# Create Master Dark

We first create the master dark for the science target. The name of the output master dark, N20120102S0538_dark.fits, is written to the screen at the end of the process.

In [ ]:
reduce_darks = Reduce()
reduce_darks.files.extend(darks60)
reduce_darks.runr()

<a class="anchor" id="Master_Flat"></a>
# Create Master Flat Field

A GNIRS master flat is created from a series of lamp-on and lamp-off exposures. Each flavor is stacked, then the lamp-off stack is subtracted from the lamp-on stack.

In [ ]:
reduce_flats = Reduce()
reduce_flats.files.extend(flats)
reduce_flats.runr()

**The default for GNIRS keyhole images is “within 600 seconds”. This can be seen by using “showpars”:**

In [ ]:
!showpars raw/N20120117S0014.fits associateSky

<a class="anchor" id="Reduce_Science"></a>
# Reduce Science Images

The science target is a point source. The sequence dithers on-target, moving the source across the thin keyhole aperture. The sky frames for each science image will be the adjacent dithered frames obtained within a certain time limit. The default for GNIRS keyhole images is “within 600 seconds”.

Both the master dark and the master flat are in our local calibration database. For any other Gemini facility instrument, they would both be retrieved automatically by the calibration manager. However, GNIRS not being an imager, and the keyhole being normally used only for acquisition, it turns out that there are no calibration association rules between GNIRS keyhole images and darks. 

The output stack units are in electrons (header keyword BUNIT=electrons). The output stack is stored in a multi-extension FITS (MEF) file. The science signal is in the “SCI” extension, the variance is in the “VAR” extension, and the data quality plane (mask) is in the “DQ” extension.

In [ ]:
reduce_target = Reduce()
reduce_target.files.extend(target)
reduce_target.uparms = [('darkCorrect:dark', 'N20120102S0538_dark.fits')]
reduce_target.runr()

<a class="anchor" id="Display_Image"></a>
# Display the Stacked Image

In [ ]:
image_file = "N20120117S0014_image.fits"
hdu_list = fits.open(image_file)
hdu_list.info()

In [ ]:
image_data = fits.getdata(image_file, ext=1)
print(image_data.shape)

In [ ]:
plt.figure(figsize = (15,15))
plt.imshow(image_data,cmap='bone',norm=Normalize(vmin=1, vmax=500),origin='lower')
plt.show()

<a class="anchor" id="Clean-up"></a>
# Optional: remove raw data (uncomment lines before running)

In [ ]:
#!rm -rf raw/